##Logistic Regression Tensorflow

In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import numpy as np

2 Loading and Preparing the MNIST Data Set

In [2]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convert to float32.

x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

# Flatten images to 1-D vector of 784 features (28*28).
num_features=784
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])

# Normalize images value from [0, 255] to [0, 1].

x_train, x_test = x_train / 255., x_test / 255.

11501568/11490434 [==============================] - 0s 0us/step


3 Setting Up Hyperparameters and Data Set Parameters

In [3]:
# MNIST dataset parameters.
num_classes = 10 # 0 to 9 digits
num_features = 784 # 28*28
# Training parameters.
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50

4 Shuffling and Batching the Data

In [4]:
# Use tf.data API to shuffle and batch data.

train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

5 Initializing Weights and Biases

In [5]:
# Weight of shape [784, 10], the 28*28 image features, and a total number of classes.
W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")

# Bias of shape [10], the total number of classes.
b = tf.Variable(tf.zeros([num_classes]), name="bias")

6 Defining Logistic Regression and Cost Function

In [6]:
# Logistic regression (Wx + b).
def logistic_regression(x):
  # Apply softmax to normalize the logits to a probability distribution.
    return tf.nn.softmax(tf.matmul(x, W) + b)

# Cross-Entropy loss function
def cross_entropy(y_pred, y_true):
    # Encode label to a one hot vector.
    y_true = tf.one_hot(y_true, depth=num_classes)

    # Clip prediction values to avoid log(0) error.
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)

    # Compute cross-entropy.
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

7 Defining Optimizers and Accuracy Metrics

In [7]:
# Accuracy metric.
def accuracy(y_pred, y_true):

# Predicted class is the index of the highest score in prediction vector (i.e. argmax).
  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.SGD(learning_rate)

8 Optimization Process and Updating Weights and Biases

In [8]:
# Optimization process.
def run_optimization(x, y):

# Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)

    # Compute gradients.
    gradients = g.gradient(loss, [W, b])

  

    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, [W, b]))

9 The Training Loop

In [9]:
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):

    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)
    if step % display_step == 0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 50, loss: 83.917328, accuracy: 0.898438
step: 100, loss: 122.090408, accuracy: 0.855469
step: 150, loss: 83.060532, accuracy: 0.917969
step: 200, loss: 60.105988, accuracy: 0.921875
step: 250, loss: 95.764908, accuracy: 0.875000
step: 300, loss: 190.718384, accuracy: 0.843750
step: 350, loss: 82.869995, accuracy: 0.937500
step: 400, loss: 457.332642, accuracy: 0.804688
step: 450, loss: 57.373962, accuracy: 0.941406
step: 500, loss: 59.072987, accuracy: 0.914062
step: 550, loss: 65.845634, accuracy: 0.933594
step: 600, loss: 333.008728, accuracy: 0.722656
step: 650, loss: 91.284149, accuracy: 0.921875
step: 700, loss: 36.801952, accuracy: 0.957031
step: 750, loss: 44.390068, accuracy: 0.941406
step: 800, loss: 71.033966, accuracy: 0.921875
step: 850, loss: 57.851971, accuracy: 0.929688
step: 900, loss: 64.505096, accuracy: 0.937500
step: 950, loss: 91.217461, accuracy: 0.941406
step: 1000, loss: 45.566856, accuracy: 0.949219


10 Testing Model Accuracy Using the Test Data

In [10]:
# Test model on validation set.
pred = logistic_regression(x_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))

Test Accuracy: 0.920900
